# 02 - Análise de Dados

Este notebook busca consumir os dados gerados das notas obtidas nas graduações e na pós-graduação para gerar os gráficos disponibilizados na página.

## Importações

In [362]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

## Constantes e sets

In [363]:
GRADES_COLOR_MAP = {
    'A': '#60D394',
    'B': '#AAF683',
    'C': '#FFD97D',
    'D': '#FF9B85',
    'F': '#EE6055' 
}

HEATMAP_PALETTE = 'RdYlGn'

# pio.templates.default = 'plotly'  # Padrão
pio.templates.default = 'plotly_white'  # Fundo branco. Melhor para embarcar no html?


## Scripts

### Análise dos dados da UFABC

Como primeiro passo, vamos fazer a análise para os dados da UFABC. Vamos primeiro olhar para as minhas notas (obtidas pela consulta ao Sigaa) e depois para as notas médias dos alunos que seguiram a mesma trajetória.

In [364]:
df_ufabc = pd.read_csv('./data/notas-ufabc.csv', dtype={'Ano': str}, sep=';')
df_ufabc

,Ano,Código,Disciplina,Resultado,Situação
0,2017.2,BIL0304-15,EVOLUÇÃO E DIVERSIFICAÇÃO DA VIDA NA TERRA,C,APROVADO
1,2017.2,BCS0001-15,BASE EXPERIMENTAL DAS CIÊNCIAS NATURAIS,A,APROVADO
2,2017.2,BIS0005-15,BASES COMPUTACIONAIS DA CIÊNCIA,A,APROVADO
3,2017.2,BIK0102-15,ESTRUTURA DA MATÉRIA,A,APROVADO
4,2017.2,BIS0003-15,BASES MATEMÁTICAS,C,APROVADO
...,...,...,...,...,...
73,2023.2,ESTA017-17,LABORATÓRIO DE MÁQUINAS ELÉTRICAS,A,APROVADO
74,2023.2,ESTA011-17,AUTOMAÇÃO DE SISTEMAS INDUSTRIAIS,A,APROVADO
75,2023.3,ESTA904-17,TRABALHO DE GRADUAÇÃO III EM ENGENHARIA DE INS...,A,APROVADO
76,2023.3,ESTA022-17,TEORIA DE ACIONAMENTOS ELÉTRICOS,A,APROVADO


Como primeiro passo, vamos verificar a proporção de notas e vamos fazer uma comparação da evolução desta proporção quadrimestre a quadrimestre. 

In [365]:
ufabc_grade_prop = df_ufabc['Resultado'].value_counts(normalize=True).reindex(['A', 'B', 'C', 'D', 'F'], fill_value=0.0)  # Para garantir que tenha todas as notas
ufabc_grade_prop

Resultado
A    0.717949
B    0.243590
C    0.038462
D    0.000000
F    0.000000
Name: proportion, dtype: float64

In [366]:
x_grades_prop_bar = ufabc_grade_prop.index
y_grades_prop_bar = ufabc_grade_prop.values
grades_prop_bar_colors = [GRADES_COLOR_MAP[grade] for grade in x_grades_prop_bar]

fig_grades_prop_bar = go.Figure()
fig_grades_prop_bar.add_trace(go.Bar(x=x_grades_prop_bar, y=y_grades_prop_bar, marker_color=grades_prop_bar_colors))
fig_grades_prop_bar.update_layout({'title': 'Proporção de notas na graduação', 'yaxis_range': [0, 1], 
                                   'yaxis_tickformat': '.0%', 'xaxis_title': 'Conceitos'})
fig_grades_prop_bar.show()

Podemos, também, fazer uma representação em gráfico de pizza.

In [367]:
fig_grades_prop_pie = go.Figure()
fig_grades_prop_pie.add_trace(go.Pie(labels=x_grades_prop_bar, values=y_grades_prop_bar, marker_colors=grades_prop_bar_colors,
                                     hoverinfo='label+percent'))
fig_grades_prop_pie.update_layout({'title': 'Proporção de notas na graduação', 'yaxis_range': [0, 1], 
                                   'yaxis_tickformat': '.0%', 'xaxis_title': 'Conceitos'})
fig_grades_prop_pie.show()

Se quisermos fazer uma comparação quadrimestre a quadrimestre, uma das possibilidades é traçar um mapa de calor.

In [368]:
df_ufabc_grades_prop_quad = df_ufabc.groupby('Ano', as_index=False).agg(
    prop_A=('Resultado', lambda s: s.value_counts(normalize=True).get('A', default=0.0)),
    prop_B=('Resultado', lambda s: s.value_counts(normalize=True).get('B', default=0.0)),
    prop_C=('Resultado', lambda s: s.value_counts(normalize=True).get('C', default=0.0)),
    prop_D=('Resultado', lambda s: s.value_counts(normalize=True).get('D', default=0.0)),
    prop_F=('Resultado', lambda s: s.value_counts(normalize=True).get('F', default=0.0))
)

df_ufabc_grades_prop_quad = df_ufabc_grades_prop_quad.sort_values(by='Ano', ascending=True).set_index('Ano', drop=True)
df_ufabc_grades_prop_quad

,prop_A,prop_B,prop_C,prop_D,prop_F
Ano,,,,,
2017.2,0.666667,0.000000,0.333333,0.0,0.0
2017.3,1.000000,0.000000,0.000000,0.0,0.0
2018.1,0.500000,0.500000,0.000000,0.0,0.0
2018.2,0.500000,0.500000,0.000000,0.0,0.0
2018.3,0.750000,0.250000,0.000000,0.0,0.0
2019.1,0.400000,0.600000,0.000000,0.0,0.0
2019.2,0.500000,0.500000,0.000000,0.0,0.0
2019.3,0.400000,0.400000,0.200000,0.0,0.0
2020.3,0.666667,0.333333,0.000000,0.0,0.0


In [369]:
x_grades_prop_quad = [f'Prop. {grade[-1]}' for grade in df_ufabc_grades_prop_quad.columns]
y_grades_prop_quad = df_ufabc_grades_prop_quad.index
z_grades_prop_quad = df_ufabc_grades_prop_quad.values

fig_grades_prop_quad = go.Figure()
fig_grades_prop_quad.add_trace(go.Heatmap(x=x_grades_prop_quad, y=y_grades_prop_quad, z=z_grades_prop_quad, text=z_grades_prop_quad,
                                          texttemplate='%{text:.1%}',  zhoverformat='.1%', colorscale=HEATMAP_PALETTE, 
                                          hoverinfo='x+y+z', colorbar_tickformat='.0%', zmin=0, zmax=1))
fig_grades_prop_quad.update_layout({'title': 'Proporção de notas quadrimestre a quadrimestre na graduação', 'height': 800})
fig_grades_prop_quad.show()

Além disso, podemos trazer uma visão acumulada, vendo como a proporção acumulada alterou-se ano após ano.

In [370]:
df_ufabc_grades_prop_cum_quad = df_ufabc.groupby('Ano', as_index=False).agg(
    prop_cum_A=('Resultado', lambda s: s.value_counts().get('A', default=0)),
    prop_cum_B=('Resultado', lambda s: s.value_counts().get('B', default=0)),
    prop_cum_C=('Resultado', lambda s: s.value_counts().get('C', default=0)),
    prop_cum_D=('Resultado', lambda s: s.value_counts().get('D', default=0)),
    prop_cum_F=('Resultado', lambda s: s.value_counts().get('F', default=0)),
)

df_ufabc_grades_prop_cum_quad = df_ufabc_grades_prop_cum_quad.sort_values(by='Ano', ascending=True).set_index('Ano', drop=True)
df_ufabc_grades_prop_cum_quad = df_ufabc_grades_prop_cum_quad.cumsum()
df_ufabc_grades_prop_cum_quad = df_ufabc_grades_prop_cum_quad.apply(lambda s: s/s.sum(), axis=1)
df_ufabc_grades_prop_cum_quad

,prop_cum_A,prop_cum_B,prop_cum_C,prop_cum_D,prop_cum_F
Ano,,,,,
2017.2,0.666667,0.000000,0.333333,0.0,0.0
2017.3,0.800000,0.000000,0.200000,0.0,0.0
2018.1,0.714286,0.142857,0.142857,0.0,0.0
2018.2,0.666667,0.222222,0.111111,0.0,0.0
2018.3,0.681818,0.227273,0.090909,0.0,0.0
2019.1,0.629630,0.296296,0.074074,0.0,0.0
2019.2,0.606061,0.333333,0.060606,0.0,0.0
2019.3,0.578947,0.342105,0.078947,0.0,0.0
2020.3,0.585366,0.341463,0.073171,0.0,0.0


In [371]:
x_grades_prop_cum_quad = [f'Prop. {grade[-1]}' for grade in df_ufabc_grades_prop_cum_quad.columns]
y_grades_prop_cum_quad = df_ufabc_grades_prop_cum_quad.index
z_grades_prop_cum_quad = df_ufabc_grades_prop_cum_quad.values

fig_grades_prop_cum_quad = go.Figure()
fig_grades_prop_cum_quad.add_trace(go.Heatmap(x=x_grades_prop_cum_quad, y=y_grades_prop_cum_quad, z=z_grades_prop_cum_quad, 
                                              text=z_grades_prop_cum_quad, texttemplate='%{text:.1%}', zhoverformat='.1%', 
                                              colorscale=HEATMAP_PALETTE, hoverinfo='x+y+z', zmin=0, zmax=1,
                                              colorbar_tickformat='.0%'))
fig_grades_prop_cum_quad.update_layout({'title': 'Proporção cumulativa de notas quadrimestre a quadrimestre na graduação', 
                                        'height': 800})
fig_grades_prop_cum_quad.show()

Para aprofundarmos a análise, vamos levar em conta agora o CR. Como esta informação não está disponível no Sigaa, vamos consultar uma base externa do [Catálogo de Disciplinas da UFABC](https://prograd.ufabc.edu.br/catalogos-de-disciplinas) (em específico a [edição de 2017](https://prograd.ufabc.edu.br/pdf/catalogo_disciplinas_graduacao_2017_2018_v2.xlsx)). Assim, vamos realizar a leitura dos dados.

In [372]:
df_ufabc_creds = pd.read_excel('./data/catalogo_disciplinas_graduacao_2017_2018_v2.xlsx')
df_ufabc_creds

,SIGLA,DISCIPLINA,TPI,RECOMENDAÇÃO,OBJETIVOS,EMENTA,BIBLIOGRAFIA BÁSICA,BIBLIOGRAFIA COMPLEMENTAR
0,ESHR022-14,Abordagens Tradicionais das Relações Internaci...,4-0-4,Não há,NaN,Contextualização histórica da emergência das t...,"CARR, Edward Hallett. Vinte anos de crise 1919...","ARON, Raymond. Paz e Guerra entre as Nações. S..."
1,ESZM035-17,Aditivação de Polímeros,4-0-4,Síntese de Polímeros; Materiais Poliméricos,Adquirir habilidades sobre o entendimento dos ...,Tipos de aditivos e métodos para obtenção de f...,"BART, J.C.J. Additives in Polymer: industrial ...","CANEVAROLO JR, S. V., Ciência dos Polímeros, A..."
2,ESZP041-14,Administração Pública e Reforma do Estado em P...,4-0-4,Não há,A disciplina visa apresentar aos alunos a vari...,"Estado, política e administração pública; Cris...","BRESSER-PEREIRA, L. C. (1998). Reforma do esta...","ABRUCIO, Fernando Luiz; LOUREIRO, Maria Rita (..."
3,ESTS016-17,Aerodinâmica I,4-0-5,Dinâmica de Gases,Familiarizar o aluno com a física associada à ...,Força de Sustenção e arrasto; Teoria do perfil...,"ANDERSON, J. D. Fundamentals of Aerodynamics. ...","BARNARD, R. H. Road Vehicle Aerodynamic Design..."
4,ESZS019-17,Aerodinâmica II,4-0-5,Aerodinâmica I,Familiarizar o aluno com a física de escoament...,Física do escoamento subsônico e hipersônico. ...,ANDERSON J. D. Hypersonic and High Temperature...,"CHATTOT, J. J. Computational Aerodynamics and ..."
...,...,...,...,...,...,...,...,...
1183,ESZA019-17,Visão Computacional,3-1-4,Fundamentos de Robótica,Compreender como se realizam diversas possibil...,Formação da imagem; extração de atributos; vis...,"BORENSTEIN, J.; EVERETT, H. R.; FENG, Liqang; ...","JONES, Joseph L. Mobile Robots - Inspiration t..."
1184,MCZA031-13,Web Semântica,4-0-4,Inteligência Artificial,NaN,Introdução à Web Semântica (WS). Linguagens pa...,"HITZLER, P., KRÖTZSCH, M., RUDOLPH, S. Foundat...","ANTONIOU, G.; GROTH, P.; VAN HARMELEN, F.; HOE..."
1185,NHT1063-15,Zoologia de Invertebrados I,2-4-3,Sistemática e Biogeografia,NaN,Fundamentos de sistemática; Origem de Metazoa ...,"BRUSCA, Richard C.; BRUSCA, Gary J. Invertebra...","AMORIM, Dalton de Souza. Fundamentos de sistem..."
1186,NHT1064-15,Zoologia de Invertebrados II,2-4-3,Sistemática e Biogeografia; Zoologia de Invert...,NaN,Plano-básico de Deuterostomia; Filogenia de Ec...,"BRUSCA, Richard C.; BRUSCA, Gary J. Invertebra...","AMORIM, Dalton de Souza. Fundamentos de sistem..."


Para a nossa análise, só será necessário ter conhecimento da sigla e do TPI, informação essa que traz o número de créditos (soma de T e P).

In [373]:
df_ufabc_creds = df_ufabc_creds[['SIGLA', 'TPI']]
df_ufabc_creds

,SIGLA,TPI
0,ESHR022-14,4-0-4
1,ESZM035-17,4-0-4
2,ESZP041-14,4-0-4
3,ESTS016-17,4-0-5
4,ESZS019-17,4-0-5
...,...,...
1183,ESZA019-17,3-1-4
1184,MCZA031-13,4-0-4
1185,NHT1063-15,2-4-3
1186,NHT1064-15,2-4-3


Feito isso, podemos juntar as bases e ver se há alguma matéria que não teve correspondência.

In [374]:
df_ufabc = df_ufabc.join(df_ufabc_creds.set_index('SIGLA'), on='Código', how='left')
df_ufabc

,Ano,Código,Disciplina,Resultado,Situação,TPI
0,2017.2,BIL0304-15,EVOLUÇÃO E DIVERSIFICAÇÃO DA VIDA NA TERRA,C,APROVADO,3-0-4
1,2017.2,BCS0001-15,BASE EXPERIMENTAL DAS CIÊNCIAS NATURAIS,A,APROVADO,0-3-2
2,2017.2,BIS0005-15,BASES COMPUTACIONAIS DA CIÊNCIA,A,APROVADO,0-2-2
3,2017.2,BIK0102-15,ESTRUTURA DA MATÉRIA,A,APROVADO,3-0-4
4,2017.2,BIS0003-15,BASES MATEMÁTICAS,C,APROVADO,4-0-5
...,...,...,...,...,...,...
73,2023.2,ESTA017-17,LABORATÓRIO DE MÁQUINAS ELÉTRICAS,A,APROVADO,0-2-4
74,2023.2,ESTA011-17,AUTOMAÇÃO DE SISTEMAS INDUSTRIAIS,A,APROVADO,1-3-4
75,2023.3,ESTA904-17,TRABALHO DE GRADUAÇÃO III EM ENGENHARIA DE INS...,A,APROVADO,0-2-4
76,2023.3,ESTA022-17,TEORIA DE ACIONAMENTOS ELÉTRICOS,A,APROVADO,4-0-4


In [375]:
df_ufabc[df_ufabc.isna().any(axis=1)]

,Ano,Código,Disciplina,Resultado,Situação,TPI


In [376]:
df_ufabc['Créditos'] = df_ufabc['TPI'].apply(lambda s: int(s.split('-')[0]) + int(s.split('-')[1]))
df_ufabc

,Ano,Código,Disciplina,Resultado,Situação,TPI,Créditos
0,2017.2,BIL0304-15,EVOLUÇÃO E DIVERSIFICAÇÃO DA VIDA NA TERRA,C,APROVADO,3-0-4,3
1,2017.2,BCS0001-15,BASE EXPERIMENTAL DAS CIÊNCIAS NATURAIS,A,APROVADO,0-3-2,3
2,2017.2,BIS0005-15,BASES COMPUTACIONAIS DA CIÊNCIA,A,APROVADO,0-2-2,2
3,2017.2,BIK0102-15,ESTRUTURA DA MATÉRIA,A,APROVADO,3-0-4,3
4,2017.2,BIS0003-15,BASES MATEMÁTICAS,C,APROVADO,4-0-5,4
...,...,...,...,...,...,...,...
73,2023.2,ESTA017-17,LABORATÓRIO DE MÁQUINAS ELÉTRICAS,A,APROVADO,0-2-4,2
74,2023.2,ESTA011-17,AUTOMAÇÃO DE SISTEMAS INDUSTRIAIS,A,APROVADO,1-3-4,4
75,2023.3,ESTA904-17,TRABALHO DE GRADUAÇÃO III EM ENGENHARIA DE INS...,A,APROVADO,0-2-4,2
76,2023.3,ESTA022-17,TEORIA DE ACIONAMENTOS ELÉTRICOS,A,APROVADO,4-0-4,4


Feito isso, agora podemos traçar a evolução temporal do CR (Coeficiente de Rendimento) ao longo dos quadrimestres. Este valor é calculado como uma média ponderada da equivalência numérica da nota pela quantidade de créditos:

$$
CR = \frac{\sum_i \text{nota}_i \cdot \text{créditos}_i}{\sum_i \text{créditos}_i}
$$

In [377]:
grades_map = {  # Mapeamento numérico da nota para número
    'A': 4,
    'B': 3,
    'C': 2,
    'D': 1,
    'F': 0
}

quad_map = {
    '1': 1,  # Q1 -> 01 -> 04 (Janeiro até Abril)
    '2': 5,  # Q2 -> 05 -> 08 (Maio -> Agosto)
    '3': 9   # Q3 -> 09 -> 12 (Setembro -> Dezembro)
}

cr_series = df_ufabc.copy().dropna()
cr_series['Nota ponderada'] = cr_series.apply(lambda s: s['Créditos'] * grades_map[s['Resultado']], axis=1)
cr_series = cr_series.groupby('Ano', as_index=False)[['Créditos', 'Nota ponderada']].sum()
cr_series[['Créditos acum', 'Nota ponderada acum']] = cr_series[['Créditos', 'Nota ponderada']].cumsum()
cr_series['CR'] = cr_series.apply(lambda s: s['Nota ponderada acum']/s['Créditos acum'], axis=1)
cr_series['Quadrimestre'] = cr_series['Ano'].apply(lambda s: pd.to_datetime(f'{quad_map[s.split('.')[1]]}/{s.split('.')[0]}')) 
cr_series = cr_series[['Quadrimestre', 'CR']]
cr_series

,Quadrimestre,CR
0,2017-05-01,3.176471
1,2017-09-01,3.562500
2,2018-01-01,3.540000
3,2018-05-01,3.553846
4,2018-09-01,3.602410
5,2019-01-01,3.565657
6,2019-05-01,3.551724
7,2019-09-01,3.496241
8,2020-09-01,3.503497
9,2021-01-01,3.531250


In [378]:
fig_cr_series = go.Figure()
fig_cr_series.add_trace(go.Scatter(x=cr_series['Quadrimestre'], y=cr_series['CR'], line_shape='spline'))
fig_cr_series.add_hline(y=4, line_color='green', line_dash='dash', annotation_text='Valor máximo')
fig_cr_series.add_hline(y=0, line_color='red', line_dash='dash', annotation_text='Valor mínimo')
fig_cr_series.update_layout({'title': 'Evolução temporal do CR', 'yaxis_range': [-0.25, 4.25], 
                             'xaxis_range': ['2017-04-01', '2024-01-01']})
fig_cr_series.show()

Além disso, conseguimos comparar o desempenho com base nos valores médios dos demais alunos por meio da base de dados obtida pelo scraping do next.

In [379]:
df_next = pd.read_csv('./data/notas-next.csv', dtype={'Ano': str}, sep=';')
df_next

,Disciplina,A,B,C,D,F,Nota provável,Conceito provável,Conceito moda
0,EVOLUÇÃO E DIVERSIFICAÇÃO DA VIDA NA TERRA,31.0,43.1,17.4,4.8,3.7,2.929000,B,B
1,BASE EXPERIMENTAL DAS CIÊNCIAS NATURAIS,84.6,15.4,0.0,0.0,0.0,3.846000,A,A
2,BASES COMPUTACIONAIS DA CIÊNCIA,39.2,26.9,18.3,7.2,8.4,2.813000,B,A
3,ESTRUTURA DA MATÉRIA,17.5,29.5,31.4,10.4,11.3,2.314685,C,C
4,BASES MATEMÁTICAS,8.7,15.4,25.9,15.5,34.5,1.483000,D,F
...,...,...,...,...,...,...,...,...,...
73,AUTOMAÇÃO DE SISTEMAS INDUSTRIAIS,36.4,35.9,19.0,5.2,3.5,2.965000,B,A
74,TRABALHO DE GRADUAÇÃO III EM ENGENHARIA DE INS...,60.0,26.7,0.0,0.0,13.3,3.201000,A,A
75,TEORIA DE ACIONAMENTOS ELÉTRICOS,26.1,30.4,22.1,9.3,12.0,2.493493,B,B
76,SISTEMAS DE CONTROLE II,6.4,32.1,34.6,15.4,11.6,2.062937,C,C


Para facilitar, vamos cruzar com a base de dados do Sigaa para obter os códigos e o CR.

In [380]:
df_next = df_next.join(df_ufabc.set_index('Disciplina')[['Código', 'Créditos', 'Ano']], on='Disciplina', how='left')
df_next

,Disciplina,A,B,C,D,F,Nota provável,Conceito provável,Conceito moda,Código,Créditos,Ano
0,EVOLUÇÃO E DIVERSIFICAÇÃO DA VIDA NA TERRA,31.0,43.1,17.4,4.8,3.7,2.929000,B,B,BIL0304-15,3,2017.2
1,BASE EXPERIMENTAL DAS CIÊNCIAS NATURAIS,84.6,15.4,0.0,0.0,0.0,3.846000,A,A,BCS0001-15,3,2017.2
2,BASES COMPUTACIONAIS DA CIÊNCIA,39.2,26.9,18.3,7.2,8.4,2.813000,B,A,BIS0005-15,2,2017.2
3,ESTRUTURA DA MATÉRIA,17.5,29.5,31.4,10.4,11.3,2.314685,C,C,BIK0102-15,3,2017.2
4,BASES MATEMÁTICAS,8.7,15.4,25.9,15.5,34.5,1.483000,D,F,BIS0003-15,4,2017.2
...,...,...,...,...,...,...,...,...,...,...,...,...
73,AUTOMAÇÃO DE SISTEMAS INDUSTRIAIS,36.4,35.9,19.0,5.2,3.5,2.965000,B,A,ESTA011-17,4,2023.2
74,TRABALHO DE GRADUAÇÃO III EM ENGENHARIA DE INS...,60.0,26.7,0.0,0.0,13.3,3.201000,A,A,ESTA904-17,2,2023.3
75,TEORIA DE ACIONAMENTOS ELÉTRICOS,26.1,30.4,22.1,9.3,12.0,2.493493,B,B,ESTA022-17,4,2023.3
76,SISTEMAS DE CONTROLE II,6.4,32.1,34.6,15.4,11.6,2.062937,C,C,ESTA008-17,5,2023.3


Antes de mais nada, vamos verificar se tivemos algum erro na junção.

In [381]:
df_next[df_next.isna().any(axis=1)]

,Disciplina,A,B,C,D,F,Nota provável,Conceito provável,Conceito moda,Código,Créditos,Ano


Com isso, agora podemos traçar os mesmos gráficos e fazer as análises.

In [382]:
# TODO

#### CIn-UFPE

In [383]:
# TODO

---

## Playground

In [384]:
df_tmp = df_ufabc.groupby('Ano', as_index=False).agg(
    prop_A=('Resultado', lambda s: s.value_counts(normalize=True).get('A', default=0.0)),
    prop_B=('Resultado', lambda s: s.value_counts(normalize=True).get('B', default=0.0)),
    prop_C=('Resultado', lambda s: s.value_counts(normalize=True).get('C', default=0.0)),
    prop_D=('Resultado', lambda s: s.value_counts(normalize=True).get('D', default=0.0)),
    prop_F=('Resultado', lambda s: s.value_counts(normalize=True).get('F', default=0.0)),
    n_materias=('Resultado', 'count'),
    total_creditos=('Créditos', 'sum')
)

df_tmp = df_tmp.sort_values(by='Ano', ascending=True).set_index('Ano', drop=True)
df_tmp

,prop_A,prop_B,prop_C,prop_D,prop_F,n_materias,total_creditos
Ano,,,,,,,
2017.2,0.666667,0.000000,0.333333,0.0,0.0,6,17
2017.3,1.000000,0.000000,0.000000,0.0,0.0,4,15
2018.1,0.500000,0.500000,0.000000,0.0,0.0,4,18
2018.2,0.500000,0.500000,0.000000,0.0,0.0,4,15
2018.3,0.750000,0.250000,0.000000,0.0,0.0,4,18
2019.1,0.400000,0.600000,0.000000,0.0,0.0,5,16
2019.2,0.500000,0.500000,0.000000,0.0,0.0,6,17
2019.3,0.400000,0.400000,0.200000,0.0,0.0,5,17
2020.3,0.666667,0.333333,0.000000,0.0,0.0,3,10


In [385]:
go.Figure([
    go.Scatter(x=df_tmp['n_materias'], y=df_tmp['prop_A'], name='Prop. A', mode='markers', marker_color=GRADES_COLOR_MAP['A']),
    go.Scatter(x=df_tmp['n_materias'], y=df_tmp['prop_B'], name='Prop. B', mode='markers', marker_color=GRADES_COLOR_MAP['B']),
    go.Scatter(x=df_tmp['n_materias'], y=df_tmp['prop_C'], name='Prop. C', mode='markers', marker_color=GRADES_COLOR_MAP['C']),
    go.Scatter(x=df_tmp['n_materias'], y=df_tmp['prop_D'], name='Prop. D', mode='markers', marker_color=GRADES_COLOR_MAP['D']),
    go.Scatter(x=df_tmp['n_materias'], y=df_tmp['prop_F'], name='Prop. F', mode='markers', marker_color=GRADES_COLOR_MAP['F'])
])

In [386]:
from scipy.stats import spearmanr, pearsonr

# pearsonr(df_tmp['n_materias'].values, df_tmp['prop_C'].values)
spearmanr(df_tmp['n_materias'].values, df_tmp['prop_C'].values)

SignificanceResult(statistic=np.float64(0.49095091382707223), pvalue=np.float64(0.03855820985902556))

In [387]:
go.Figure([
    go.Scatter(x=df_tmp['total_creditos'], y=df_tmp['prop_A'], name='Prop. A', mode='markers', marker_color=GRADES_COLOR_MAP['A']),
    go.Scatter(x=df_tmp['total_creditos'], y=df_tmp['prop_B'], name='Prop. B', mode='markers', marker_color=GRADES_COLOR_MAP['B']),
    go.Scatter(x=df_tmp['total_creditos'], y=df_tmp['prop_C'], name='Prop. C', mode='markers', marker_color=GRADES_COLOR_MAP['C']),
    go.Scatter(x=df_tmp['total_creditos'], y=df_tmp['prop_D'], name='Prop. D', mode='markers', marker_color=GRADES_COLOR_MAP['D']),
    go.Scatter(x=df_tmp['total_creditos'], y=df_tmp['prop_F'], name='Prop. F', mode='markers', marker_color=GRADES_COLOR_MAP['F'])
])

In [388]:
from scipy.stats import spearmanr, pearsonr

# pearsonr(df_tmp['total_creditos'].values, df_tmp['prop_C'].values)
spearmanr(df_tmp['total_creditos'].values, df_tmp['prop_B'].values)

SignificanceResult(statistic=np.float64(0.07481635477231192), pvalue=np.float64(0.7679598914640993))